#Terrain Graphics

In [ ]:
#| default_exp terraingraphics

In [ ]:
from fastcore.basics import patch

In [ ]:
#| export
from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, HexRegion
from HexMagic.terrain import  TerraDemo, Terrain
from HexMagic.terrainpatterns import TerrainPatterns

In [ ]:
sampleMap = TerraDemo().sanFran()
    
sampleMap.hexGrid.adjustRadius(20)
aRender = sampleMap.hexGrid.builder
sampleMap.colorMap()

In [ ]:
@patch
def styleRegions(self:HexGrid):
    ret = {}
    for i, h in enumerate(self.hexes):
        region = ret.get(h.style.name,HexRegion(set(),self))
        region.hexes.add(i)
        ret[h.style.name] = region
    return ret

In [ ]:
sampleMap.hexGrid.styleRegions().items()

dict_items([('sealevel', HexRegion(hexes={0, 1, 2, 3, 4, 5, 6, 7, 512, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 535, 30, 31, 32, 33, 34, 35, 36, 37, 541, 542, 41, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 571, 572, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 601, 602, 629, 630, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 631, 632, 536, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 539, 540, 662, 665, 176, 177, 178, 179, 180, 181, 182, 690, 691, 692, 694, 717, 718, 207, 208, 209, 210, 211, 720, 721, 722, 725, 538, 747, 748, 237, 238, 239, 240, 241, 749, 750, 751, 752, 753, 756, 660, 661, 266, 267, 268, 269, 270, 271, 778, 779, 780, 781, 782, 783, 808, 297, 298, 299, 300, 301, 809, 569, 810, 811, 812, 813, 570, 835, 324, 325, 836, 327, 328, 329, 330, 331, 837, 838, 839, 840, 841, 842, 843, 865, 866, 355, 356, 357, 358, 359, 360, 361, 867, 868, 869, 870, 871, 872, 873, 892, 895, 896, 897, 386, 898, 

In [ ]:
@patch
def contiguous(self: HexRegion) -> list[HexRegion]:
    """Split region into contiguous sub-regions."""
    grid = self.hex_grid
    remaining = set(self.hexes)
    regions = []
    
    while remaining:
        # Start a new region with any remaining hex
        start = next(iter(remaining))
        connected = set()
        frontier = {start}
        
        # Flood fill
        while frontier:
            current = frontier.pop()
            if current in connected:
                continue
            connected.add(current)
            remaining.discard(current)
            
            # Add neighbors that are in our region
            for hp in HexPosition(0, 0, 0).ring(1):
                neighbor_idx = grid.hexposition_to_index(hp, current)
                if neighbor_idx >= 0 and neighbor_idx in remaining:
                    frontier.add(neighbor_idx)
        
        regions.append(HexRegion(connected, grid))
    
    return regions


@patch
def contiguousPath(self: HexRegion, includeEdge=True) -> list[tuple]:
    """Trace perimeter of a contiguous region, returning ordered edge tuples.
    
    Since region is contiguous, returns a single closed path of edges.
    """
    grid = self.hex_grid
    edges = []  # List of (x1, y1, x2, y2) tuples
    vertex_adj = {}
    
    def snap(v):
        return (round(v[0]), round(v[1]))
    
    # Collect all perimeter edges
    for idx in self.hexes:
        hex_obj = grid.hexes[idx]
        
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor_idx = grid.hexposition_to_index(hp, idx)
            
            if neighbor_idx < 0 or neighbor_idx not in self.hexes:
                if neighbor_idx >= 0:
                    edge = grid.commonEdge(idx, neighbor_idx)
                elif includeEdge:
                    edge = neighbor_idx.edgeFrom(dir_idx)
                    print("{neighbor_idx} {dir_idx} is {edge}")
                else:
                    continue
                
                if edge:
                    k1 = snap((edge[0], edge[1]))
                    k2 = snap((edge[2], edge[3]))
                    vertex_adj.setdefault(k1, set()).add(k2)
                    vertex_adj.setdefault(k2, set()).add(k1)
    
    if not vertex_adj:
        return []
    
    # Trace single path (contiguous = one closed loop)
    visited_edges = set()
    start = next(iter(vertex_adj.keys()))
    path_edges = []
    current = start
    
    while True:
        neighbors = vertex_adj.get(current, set())
        
        next_vertex = None
        for n in neighbors:
            edge_key = (min(current, n), max(current, n))
            if edge_key not in visited_edges:
                path_edges.append((current[0], current[1], n[0], n[1]))
                visited_edges.add(edge_key)
                next_vertex = n
                break
        
        if next_vertex is None or next_vertex == start:
            break
        current = next_vertex
    
    return path_edges


@patch
def contiguousPath(self: HexRegion, includeEdge=True) -> list[tuple]:
    """Trace perimeter, handling open paths at map boundary."""
    grid = self.hex_grid
    vertex_adj = {}
    
    def snap(v):
        return (round(v[0]), round(v[1]))
    
    for idx in self.hexes:
        hex_obj = grid.hexes[idx]
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor_idx = grid.hexposition_to_index(hp, idx)
            if neighbor_idx < 0 or neighbor_idx not in self.hexes:
                if neighbor_idx >= 0:
                    edge = grid.commonEdge(idx, neighbor_idx)
                elif includeEdge:
                    edge = hex_obj.edgeFrom(dir_idx)
                else:
                    continue
                if edge:
                    k1 = snap((edge[0], edge[1]))
                    k2 = snap((edge[2], edge[3]))
                    vertex_adj.setdefault(k1, []).append(k2)
                    vertex_adj.setdefault(k2, []).append(k1)
    
    if not vertex_adj:
        return []
    
    # Find a degree-1 vertex to start (boundary case) or any vertex (closed loop)
    start = None
    for v, adj in vertex_adj.items():
        if len(adj) == 1:
            start = v
            break
    if start is None:
        start = next(iter(vertex_adj.keys()))
    
    # Trace path, consuming edges
    path_edges = []
    current = start
    visited_edges = set()
    
    while True:
        adj = vertex_adj.get(current, [])
        next_vertex = None
        for n in adj:
            edge_key = (min(current, n), max(current, n))
            if edge_key not in visited_edges:
                path_edges.append((current[0], current[1], n[0], n[1]))
                visited_edges.add(edge_key)
                next_vertex = n
                break
        
        if next_vertex is None:
            break
        current = next_vertex
    
    return path_edges


@patch
def simplePath(self:HexRegion, edges,style):
    start_x = edges[0][0]
    start_y = edges[0][1]

    ret = [MapCord(start_x, start_y)]
    for edge in edges:
        if start_x == edge[0] and start_y == edge[1]:
            start_x = edge[2]
            start_y = edge[3]
        else:
            start_x = edge[0]
            start_y = edge[1]
        ret.append(MapCord(start_x, start_y))
    return MapPath(ret, style=style)

@patch
def from_boundary(self: HexRegion, points: list[tuple[float, float]], style=None) -> 'MapPath':
        """Factory method to create a closed boundary path from coordinate tuples.

        Args:
            points: List of (x, y) tuples
            style: Optional StyleCSS, defaults to blank

        Returns:
            MapPath with points converted to MapCord
        """
        if style is None:
            style = StyleCSS("blank")
        path = MapPath(points, style)
        return path.closed()  # Use existing closed() method

In [ ]:
#| export
@patch
def trace_boundary(self: HexRegion) -> list[tuple[int, int]]:
    """Trace boundary as (hex_index, vertex_index) pairs.
    
    Walks clockwise around perimeter. Returns list of (hex_idx, vertex_idx).
    """
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    
    # Find a boundary hex: one with at least one non-region neighbor
    start_hex = None
    start_vertex = None
    
    for idx in self.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                # This edge is on boundary - start here
                start_hex = idx
                # Get first vertex of this edge (clockwise)
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []  # No boundary (shouldn't happen)
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        # Next vertex clockwise on this hex
        next_vertex = (current_vertex + 1) % 6
        
        # Which direction does edge (current_vertex, next_vertex) face?
        # Find which direction has these vertices
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            # Check if neighbor across this edge is in region
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            if neighbor >= 0 and neighbor in self.hexes:
                # Jump to neighbor - enter at corresponding vertex
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
            else:
                # Stay on this hex, advance vertex
                current_vertex = next_vertex
        else:
            # Edge not found (shouldn't happen), just advance
            current_vertex = next_vertex
        
        # Check if back to start
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        # Safety: prevent infinite loop
        if len(path) > len(self.hexes) * 6:
            print(f"Warning: path too long, breaking")
            break
    
    return path

@patch
def boundary_to_coords(self: HexRegion, path: list[tuple[int, int]]) -> list[MapCord]:
    """Convert (hex_index, vertex_index) path to MapCord list."""
    return [self.hex_grid.hexes[hex_idx].v[vertex_idx] for hex_idx, vertex_idx in path]

@patch 
def boundary_path(self: HexRegion, style=None) -> MapPath:
    """Get boundary as a MapPath ready for rendering."""
    if style is None:
        style = StyleCSS("boundary", fill="none", stroke="#333", stroke_width=2)
    
    path = self.trace_boundary()
    coords = self.boundary_to_coords(path)
    return MapPath(coords, style).closed()



@patch
def trace_boundary(self: HexRegion) -> list[tuple[int, int]]:
    """Trace boundary as (hex_index, vertex_index) pairs."""
    if not self.hexes:
        return []
    
    # Special case: single hex region
    if len(self.hexes) == 1:
        hex_idx = next(iter(self.hexes))
        # Return all 6 vertices in order
        return [(hex_idx, v) for v in range(6)]
    
    # ... rest of existing multi-hex logic ...
    grid = self.hex_grid
    
    # Find a boundary hex: one with at least one non-region neighbor
    start_hex = None
    start_vertex = None
    
    for idx in self.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []

    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        # Next vertex clockwise on this hex
        next_vertex = (current_vertex + 1) % 6
        
        # Which direction does edge (current_vertex, next_vertex) face?
        # Find which direction has these vertices
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            # Check if neighbor across this edge is in region
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            if neighbor >= 0 and neighbor in self.hexes:
                # Jump to neighbor - enter at corresponding vertex
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
            else:
                # Stay on this hex, advance vertex
                current_vertex = next_vertex
        else:
            # Edge not found (shouldn't happen), just advance
            current_vertex = next_vertex
        
        # Check if back to start
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        # Safety: prevent infinite loop
        if len(path) > len(self.hexes) * 6:
            print(f"Warning: path too long, breaking")
            break
    
    return path

In [ ]:
#| export
@patch
def styleLayer(self:HexGrid):
    retLayer = ""
    regions = self.styleRegions()
    for styleName, region in regions.items():
        style = self.builder.styles[styleName]
        for subR in region.contiguous():
            path = subR.boundary_path(style=style)
            retLayer += path.drawPloy()
    return retLayer

aRender.adjust("regions", sampleMap.hexGrid.styleLayer())
aRender.show()


In [ ]:
from HexMagic.voronoi import generate_plate_terrain

Terrain.fromSeeds = generate_plate_terrain

In [ ]:
mySize = MapSize(480,480)
myBounds = MapRect(MapCord(0,0), mySize)
sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)
sampleMap.colorMap
sampleMap.hexGrid.builder.adjust("regions", sampleMap.hexGrid.styleLayer())
#isf.hexGrid.update()
sampleMap.hexGrid.builder.show()

Initial seeds: 16 (spacing=8)
After elimination: 12
Final unique seeds: 12


In [ ]:
@patch
def trace_boundary(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary as (hex_index, vertex_index) pairs.
    
    Walks clockwise around perimeter. Returns list of (hex_idx, vertex_idx).
    """
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    
    # Find a boundary hex: one with at least one non-region neighbor
    start_hex = None
    start_vertex = None
    
    for idx in self.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")

    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    iterations = 0
    
    while True:
        iterations += 1
        if debug:
            print(f"\n--- Iteration {iterations} ---")
            print(f"Current: hex {current_hex}, vertex {current_vertex}")
        
        path.append((current_hex, current_vertex))
        
        # Next vertex clockwise on this hex
        next_vertex = (current_vertex + 1) % 6
        if debug:
            print(f"Next vertex (clockwise): {next_vertex}")
        
        # Which direction does edge (current_vertex, next_vertex) face?
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            if debug:
                print(f"Edge direction: {edge_dir}")
            
            # Check if neighbor across this edge is in region
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            if debug:
                print(f"Neighbor in direction {edge_dir}: {neighbor}")
                print(f"Neighbor in region: {neighbor >= 0 and neighbor in self.hexes}")
            
            if neighbor >= 0 and neighbor in self.hexes:
                # Jump to neighbor - enter at corresponding vertex
                new_vertex = (next_vertex + 3) % 6
                if debug:
                    print(f"→ Jumping to hex {neighbor}, vertex {new_vertex}")
                current_hex = neighbor
                current_vertex = new_vertex
            else:
                # Stay on this hex, advance vertex
                if debug:
                    print(f"→ Staying on hex {current_hex}, advancing to vertex {next_vertex}")
                current_vertex = next_vertex
        else:
            if debug:
                print(f"ERROR: Edge not found for vertices {current_vertex}→{next_vertex}")
            current_vertex = next_vertex
        
        # Check if back to start
        if current_hex == start_hex and current_vertex == start_vertex:
            if debug:
                print(f"\n✓ Returned to start! Path length: {len(path)}")
            break
        
        # Safety: prevent infinite loop
        if iterations > len(self.hexes) * 6:
            print(f"⚠ Warning: path too long ({iterations} iterations), breaking")
            break
    
    return path


In [ ]:
def pompeii():
    with open("data/templates/pompeii.txt", "r") as f:
        myGrid = Terrain.decode(f.read())
    return myGrid

pomp = pompeii().scaled(.25)
pomp.colorMap()
pomp.hexGrid.update()
#pomp.hexGrid.adjustRadius(40)
pomp.hexGrid.builder.adjust("regions", pomp.hexGrid.styleLayer())

#clearStyle = StyleCSS("clear",fill=None,stroke="purple")
fill = "yellow"
hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
grid = pomp.hexGrid
      #Need to add styles as we go along
grid.builder.add_style(labelStyle)
grid.builder.add_style(hexStyle)

    #itterate through the hexes
for i in range(len(grid.hexes)):

    grid.hexes[i].label = str(i) 
    grid.hexes[i].style = hexStyle
    grid.hexes[i].labelStyle = labelStyle.name

pomp.hexGrid.update()
pomp.hexGrid.builder.show()

⚠ Warning: path too long (259 iterations), breaking


In [ ]:
pomp = pompeii().scaled(.25)
pomp.colorMap()
grid = pomp.hexGrid
land = grid.styleRegions()["Lowland"].contiguous()[0]
print(land.boundary_path().svg())



	
	<path d="M 241.0 204.0 C 241.0 204.0 241.0 204.0 241.0 204.0 C 241.0 204.0
		 241.0 204.0 241.0 204.0" class ="boundary" />
	


In [ ]:
print(land.hexes)

{128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 141, 142, 143, 11, 23, 35, 47, 59, 71, 82, 83, 84, 94, 95, 96, 97, 105, 106, 107, 109, 110, 111, 115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127}


There is something wrong with the boundary check here

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, I can see the issue. In the `boundary_path` method, you're calling `trace_boundary()` which should return a list of `(hex_index, vertex_index)` tuples. But when I look at the output, the path has only one point repeated - `(241.0, 204.0)` - which suggests the trace isn't working correctly.

The problem is likely that hex index 241 doesn't exist in your grid, or the starting hex finding logic isn't working properly for this particular region.

Let me ask: What does `len(grid.hexes)` return? And can you check if any of the hex indices in `land.hexes` (like 128, 129, etc.) are actually valid indices in your grid?

Try running:
```python
print(f"Grid has {len(grid.hexes)} hexes")
print(f"Max index in land: {max(land.hexes)}")
```

This will help us see if there's an index mismatch issue.

In [ ]:
print(f"Grid has {len(grid.hexes)} hexes")
print(f"Max index in land: {max(land.hexes)}")

Grid has 144 hexes
Max index in land: 143


In [ ]:
@patch
def trace_boundary(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary as (hex_index, vertex_index) pairs.
    
    Walks clockwise around perimeter. Returns list of (hex_idx, vertex_idx).
    """
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    
    # Find a boundary hex: one with at least one non-region neighbor
    start_hex = None
    start_vertex = None
    
    for idx in self.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    iterations = 0
    
    while True:
        iterations += 1
        if debug:
            print(f"\n--- Iteration {iterations} ---")
            print(f"Current: hex {current_hex}, vertex {current_vertex}")
        
        path.append((current_hex, current_vertex))
        
        # Next vertex clockwise on this hex
        next_vertex = (current_vertex + 1) % 6
        if debug:
            print(f"Next vertex (clockwise): {next_vertex}")
        
        # Which direction does edge (current_vertex, next_vertex) face?
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            if debug:
                print(f"Edge direction: {edge_dir}")
            
            # Check neighbor across this edge
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            in_region = neighbor >= 0 and neighbor in self.hexes
            if debug:
                print(f"Neighbor: {neighbor}, in_region: {in_region}")
            
            # KEY FIX: Only jump if this is an INTERIOR edge
            # We're on the boundary, so we should NOT jump - we stay and advance
            if in_region:
                # This is an interior edge - check if we should turn the corner
                # Look at the edge AFTER this one to see if it's also interior
                check_vertex = (next_vertex + 1) % 6
                check_edge_dir = None
                for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
                    if v1 == next_vertex and v2 == check_vertex:
                        check_edge_dir = dir_idx
                        break
                
                if check_edge_dir is not None:
                    check_hp = HexPosition.directions()[check_edge_dir]
                    check_neighbor = grid.hexposition_to_index(check_hp, current_hex)
                    check_in_region = check_neighbor >= 0 and check_neighbor in self.hexes
                    
                    if debug:
                        print(f"Next edge check: neighbor {check_neighbor}, in_region: {check_in_region}")
                    
                    if not check_in_region:
                        # Next edge is boundary, so stay on this hex
                        if debug:
                            print(f"→ Staying on hex {current_hex}, advancing to vertex {next_vertex}")
                        current_vertex = next_vertex
                    else:
                        # Next edge is also interior, jump to neighbor
                        new_vertex = (next_vertex + 3) % 6
                        if debug:
                            print(f"→ Jumping to hex {neighbor}, vertex {new_vertex}")
                        current_hex = neighbor
                        current_vertex = new_vertex
                else:
                    current_vertex = next_vertex
            else:
                # Boundary edge - stay on this hex
                if debug:
                    print(f"→ Boundary edge, staying on hex {current_hex}, advancing to vertex {next_vertex}")
                current_vertex = next_vertex
        else:
            if debug:
                print(f"ERROR: Edge not found")
            current_vertex = next_vertex
        
        # Check if back to start
        if current_hex == start_hex and current_vertex == start_vertex:
            if debug:
                print(f"\n✓ Returned to start! Path length: {len(path)}")
            break
        
        # Safety: prevent infinite loop
        if iterations > len(self.hexes) * 6:
            print(f"⚠ Warning: path too long ({iterations} iterations), breaking")
            break
    
    return path


In [ ]:
land.trace_boundary(debug=True)

Starting at hex 131, vertex 2

--- Iteration 1 ---
Current: hex 131, vertex 2
Next vertex (clockwise): 3
Edge direction: 0
Neighbor: 142, in_region: True
Next edge check: neighbor 130, in_region: True
→ Jumping to hex 142, vertex 0

--- Iteration 2 ---
Current: hex 142, vertex 0
Next vertex (clockwise): 1
Edge direction: 4
Neighbor: 143, in_region: True
Next edge check: neighbor -1, in_region: False
→ Staying on hex 142, advancing to vertex 1

--- Iteration 3 ---
Current: hex 142, vertex 1
Next vertex (clockwise): 2
Edge direction: 5
Neighbor: -1, in_region: False
→ Boundary edge, staying on hex 142, advancing to vertex 2

--- Iteration 4 ---
Current: hex 142, vertex 2
Next vertex (clockwise): 3
Edge direction: 0
Neighbor: -1, in_region: False
→ Boundary edge, staying on hex 142, advancing to vertex 3

--- Iteration 5 ---
Current: hex 142, vertex 3
Next vertex (clockwise): 4
Edge direction: 1
Neighbor: 141, in_region: True
Next edge check: neighbor 130, in_region: True
→ Jumping to hex

[(131, 2),
 (142, 0),
 (142, 1),
 (142, 2),
 (142, 3),
 (141, 1),
 (141, 2),
 (141, 3),
 (140, 1),
 (140, 2),
 (140, 3),
 (139, 1),
 (139, 2),
 (139, 3),
 (138, 1),
 (138, 2),
 (138, 3),
 (137, 1),
 (137, 2),
 (137, 3),
 (136, 1),
 (136, 2),
 (136, 3),
 (135, 1),
 (135, 2),
 (135, 3),
 (135, 4),
 (123, 2),
 (123, 3),
 (122, 1),
 (122, 2),
 (122, 3),
 (122, 4),
 (109, 2),
 (109, 3),
 (109, 4),
 (109, 5),
 (109, 0),
 (110, 4),
 (110, 5),
 (110, 0),
 (111, 4),
 (111, 5),
 (111, 0),
 (111, 1),
 (124, 5),
 (124, 0),
 (125, 4),
 (125, 5),
 (125, 0),
 (126, 4),
 (126, 5),
 (126, 0),
 (127, 4),
 (127, 5),
 (115, 3),
 (115, 4),
 (115, 5),
 (115, 0),
 (116, 4),
 (116, 5),
 (105, 3),
 (105, 4),
 (105, 5),
 (105, 0),
 (106, 4),
 (106, 5),
 (94, 3),
 (94, 4),
 (82, 2),
 (82, 3),
 (82, 4),
 (82, 5),
 (82, 0),
 (83, 4),
 (83, 5),
 (83, 0),
 (83, 1),
 (95, 5),
 (95, 0),
 (95, 1),
 (95, 2),
 (107, 0),
 (107, 1),
 (119, 5),
 (119, 0),
 (119, 1),
 (119, 2),
 (131, 0),
 (131, 1),
 (143, 5),
 (143, 0),
 (1

it got back to the start too quickly

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're right! The trace found only 3 hexes (131, 142, 143) when `land.hexes` contains 43 hexes. The algorithm is taking a shortcut and returning to the start immediately instead of walking around the entire perimeter.

The issue is in the logic for when to "jump" to a neighbor. The algorithm jumps to a neighbor whenever that neighbor is *in the region*, but it should only jump when the neighbor is in the region *and* we're walking along an internal edge (not the boundary).

The key insight: we should only stay on the current hex and advance the vertex when we're on a boundary edge (neighbor not in region OR neighbor < 0).

Do you want to try fixing the condition, or would you like me to suggest the change?

So that broke something else. so a basic sanity check is that we can find the perimeter hexes by doing an outside of an outside of a hex region to get the perimeter hexes. we can then build up the hex adjacecy to determine neighbors. we then have a few cases
1 there are no neighbors - this is just a single hex return border
2 there are one neighbors - you must leave and return via the same node
3 there are two neighbors - you leave and return verus oppostie
more than two you enter and leave in either a clockwise or counter clockwise fashion. does buidling the adjaceny graph of the perimiter hexes first help?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, that's a much cleaner approach! Building the adjacency graph of perimeter hexes first would definitely help. Let me break down why:

**Benefits:**
1. You immediately know which hexes form the boundary (those with at least one non-region neighbor)
2. You can build a graph showing how perimeter hexes connect along the boundary
3. The walking logic becomes simpler - you know exactly which edges are boundary edges

**Your case analysis is spot-on:**
- 0 neighbors: single hex, trace all 6 vertices
- 1 neighbor: enter/exit same edge (creates a "peninsula")
- 2 neighbors: typically opposite sides (straight corridor)
- 3+ neighbors: corner hexes where you turn

Would you like to try implementing the perimeter hex identification first, or should I show you a small code snippet to get started?

In [ ]:
@patch
def perimeterHexes(self:HexRegion):
    return self & (self.outside().outside())

In [ ]:
land.perimeterHexes()

HexRegion(hexes={135, 11, 23, 35, 47, 59, 71, 82, 83, 84, 94, 96, 97, 105, 106, 109, 110, 111, 115, 116, 122, 123, 124, 125, 126, 127}, hex_grid=<HexMagic.primitives.HexGrid object>)

In [ ]:
!cat ../HexMagic/primitives.py

# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/02_primitives.ipynb.

# %% auto 0
__all__ = ['PrimitiveDemo', 'MapCord', 'MapSize', 'MapRect', 'MakeCord', 'MakeSize', 'MapPath', 'HexPosition', 'GosperCurve',
           'Hex', 'hexSVG', 'hexBackground', 'HexGrid', 'HexWrapper', 'HexRegion', 'LinearGradient']

# %% ../nbs/02_primitives.ipynb 6
import numpy as np
import math


from collections import namedtuple
from dataclasses import dataclass, field
from fastcore.basics import patch
from bezier_interpolation import cubic_interpolation
from typing import List

from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# %% ../nbs/02_primitives.ipynb 7
import sys
import os
import math
import random
import traceback
from functools import lru_cache # for ring positions

from .styles import StyleCSS, SVGLayer, SVGBuilder, SVGPatternLoader, preview, indent,SVGDef, Generatable

# %% ../nbs/02_primitives.ipynb 9
class PrimitiveDemo:
    def __init__(self):
        self.help =

In [ ]:
@patch
def perimeterGraph(self:HexRegion):
    ret = {}
    perm = self.perimeterHexes()
    for x in self.perimeterHexes():
        reg = HexRegion(set([x]),self.hex_grid).outside()
        ret[x] = reg & perm
    return ret

In [ ]:
land.perimeterGraph()

{135: HexRegion(hexes={123, 124}, hex_grid=<HexMagic.primitives.HexGrid object>),
 11: HexRegion(hexes={23}, hex_grid=<HexMagic.primitives.HexGrid object>),
 23: HexRegion(hexes={35, 11}, hex_grid=<HexMagic.primitives.HexGrid object>),
 35: HexRegion(hexes={23, 47}, hex_grid=<HexMagic.primitives.HexGrid object>),
 47: HexRegion(hexes={59, 35}, hex_grid=<HexMagic.primitives.HexGrid object>),
 59: HexRegion(hexes={47, 71}, hex_grid=<HexMagic.primitives.HexGrid object>),
 71: HexRegion(hexes={83, 59}, hex_grid=<HexMagic.primitives.HexGrid object>),
 82: HexRegion(hexes={83, 94}, hex_grid=<HexMagic.primitives.HexGrid object>),
 83: HexRegion(hexes={82, 94, 71}, hex_grid=<HexMagic.primitives.HexGrid object>),
 84: HexRegion(hexes={96, 97}, hex_grid=<HexMagic.primitives.HexGrid object>),
 94: HexRegion(hexes={106, 83, 82}, hex_grid=<HexMagic.primitives.HexGrid object>),
 96: HexRegion(hexes={97, 84}, hex_grid=<HexMagic.primitives.HexGrid object>),
 97: HexRegion(hexes={96, 84, 109}, hex_grid

can you build the new function using these?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Here's a new approach using the perimeter graph:

```python
@patch
def trace_boundary_v2(self: HexRegion, style=None) -> MapPath:
    """Trace boundary using perimeter hex adjacency graph."""
    if not self.hexes:
        return MapPath([], style or StyleCSS("boundary"))
    
    if style is None:
        style = StyleCSS("boundary", fill="none", stroke="#333", stroke_width=2)
    
    grid = self.hex_grid
    perm_hexes = self.perimeterHexes()
    
    # Single hex case
    if len(perm_hexes.hexes) == 1:
        idx = next(iter(perm_hexes.hexes))
        return MapPath(grid.hexes[idx].v, style).closed()
    
    # Build adjacency graph
    adj = self.perimeterGraph()
    
    # Find boundary edges for each perimeter hex
    def get_boundary_edges(idx):
        """Return list of (dir_idx, edge) for edges facing outside region."""
        edges = []
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                edges.append(dir_idx)
        return edges
    
    # Start at a perimeter hex with fewest neighbors (likely a corner/endpoint)
    start_hex = min(adj.keys(), key=lambda x: len(adj[x].hexes))
    
    # Walk the perimeter
    visited = set()
    path_vertices = []
    current = start_hex
    prev = None
    
    while current not in visited or current == start_hex and len(visited) < len(perm_hexes.hexes):
        if current in visited:
            break
        visited.add(current)
        
        hex_obj = grid.hexes[current]
        boundary_dirs = get_boundary_edges(current)
        
        # Get vertices for boundary edges in clockwise order
        for dir_idx in sorted(boundary_dirs):
            v1_idx, v2_idx = Hex._direction_to_vertices[dir_idx]
            path_vertices.append(hex_obj.v[v1_idx])
            path_vertices.append(hex_obj.v[v2_idx])
        
        # Move to next unvisited neighbor
        neighbors = adj[current].hexes
        next_hex = None
        for n in neighbors:
            if n not in visited:
                next_hex = n
                break
        
        if next_hex is None:
            break
        prev = current
        current = next_hex
    
    # Remove duplicate consecutive vertices
    cleaned = [path_vertices[0]] if path_vertices else []
    for v in path_vertices[1:]:
        if v != cleaned[-1]:
            cleaned.append(v)
    
    return MapPath(cleaned, style).closed()
```

This is a starting point, but the vertex ordering between hexes is still tricky. Want me to refine it further, or would you prefer to work through the edge cases step by step?

so when we traverse we need to track where we entered a hex. we did this with (current_hex, current_vertex)

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Right! Let me rewrite using the perimeter graph but keeping the `(current_hex, current_vertex)` state:

```python
@patch
def trace_boundary_v2(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary using perimeter graph, tracking (hex_idx, vertex_idx)."""
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    perm_hexes = self.perimeterHexes()
    
    # Single hex - return all 6 vertices
    if len(perm_hexes.hexes) == 1:
        idx = next(iter(perm_hexes.hexes))
        return [(idx, v) for v in range(6)]
    
    adj = self.perimeterGraph()
    
    # Find starting hex and vertex (first boundary edge)
    start_hex = None
    start_vertex = None
    
    for idx in perm_hexes.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        next_vertex = (current_vertex + 1) % 6
        
        # Find which direction this edge faces
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            # Key change: only jump if neighbor is a PERIMETER hex
            if neighbor >= 0 and neighbor in perm_hexes.hexes:
                # Jump to perimeter neighbor
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
            else:
                # Stay on current hex (boundary or interior neighbor)
                current_vertex = next_vertex
        else:
            current_vertex = next_vertex
        
        # Back to start?
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        if len(path) > len(perm_hexes.hexes) * 6:
            print(f"⚠ Warning: path too long")
            break
    
    return path
```

The key change: **only jump when the neighbor is a perimeter hex**. Interior hexes don't participate in the boundary walk at all.

In [ ]:
@patch
def trace_boundary_v2(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary using perimeter graph, tracking (hex_idx, vertex_idx)."""
    if not self.hexes:
        return []
    
    print("trace_boundary_v2")
    grid = self.hex_grid
    perm_hexes = self.perimeterHexes()
    
    # Single hex - return all 6 vertices
    if len(perm_hexes.hexes) == 1:
        idx = next(iter(perm_hexes.hexes))
        return [(idx, v) for v in range(6)]
    
    adj = self.perimeterGraph()
    
    # Find starting hex and vertex (first boundary edge)
    start_hex = None
    start_vertex = None
    
    for idx in perm_hexes.hexes:
        print(f"going through perm_hexes {id}")
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        next_vertex = (current_vertex + 1) % 6
        
        # Find which direction this edge faces
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            # Key change: only jump if neighbor is a PERIMETER hex
            if neighbor >= 0 and neighbor in perm_hexes.hexes:
                # Jump to perimeter neighbor
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
            else:
                # Stay on current hex (boundary or interior neighbor)
                current_vertex = next_vertex
        else:
            current_vertex = next_vertex
        
        # Back to start?
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        if len(path) > len(perm_hexes.hexes) * 6:
            print(f"⚠ Warning: path too long")
            break
    
    return path


In [ ]:

@patch 
def boundary_path(self: HexRegion, style=None) -> MapPath:
    """Get boundary as a MapPath ready for rendering."""
    if style is None:
        style = StyleCSS("boundary", fill="none", stroke="#333", stroke_width=2)
    
    path = self.trace_boundary_v2()
    coords = self.boundary_to_coords(path)
    return MapPath(coords, style).closed()

In [ ]:
@patch
def perimeterHexes(self: HexRegion) -> HexRegion:
    """Find hexes on the perimeter (have at least one non-region or off-grid neighbor)."""
    grid = self.hex_grid
    perimeter = set()
    
    for idx in self.hexes:
        for hp in HexPosition(0, 0, 0).ring(1):
            neighbor = grid.hexposition_to_index(hp, idx)
            # Perimeter if neighbor is off-grid OR not in region
            if neighbor < 0 or neighbor not in self.hexes:
                perimeter.add(idx)
                break
    
    return HexRegion(perimeter, grid)


In [ ]:

pomp = pompeii().scaled(.25)
pomp.colorMap()
pomp.hexGrid.update()
#pomp.hexGrid.adjustRadius(40)
pomp.hexGrid.builder.adjust("regions", pomp.hexGrid.styleLayer())

#clearStyle = StyleCSS("clear",fill=None,stroke="purple")
fill = "yellow"
hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
grid = pomp.hexGrid
      #Need to add styles as we go along
grid.builder.add_style(labelStyle)
grid.builder.add_style(hexStyle)

    #itterate through the hexes
for i in range(len(grid.hexes)):

    grid.hexes[i].label = str(i) 
    grid.hexes[i].style = hexStyle
    grid.hexes[i].labelStyle = labelStyle.name

pomp.hexGrid.update()
pomp.hexGrid.builder.show()

trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
trace_boundary_v2
going through perm_hexes <built-in function id>
trace_boundary_v2
going thro

In [ ]:
@patch
def flood_fill(self: HexGrid, start_idx: int) -> HexRegion:
    """Flood fill from start_idx to all connected hexes with same style."""
    if start_idx < 0 or start_idx >= len(self.hexes):
        return HexRegion(set(), self)
    
    target_style = self.hexes[start_idx].style.name
    visited = set()
    region = set()
    frontier = {start_idx}
    
    while frontier:
        current = frontier.pop()
        if current in visited:
            continue
        visited.add(current)
        
        if self.hexes[current].style.name == target_style:
            region.add(current)
            # Add all neighbors to frontier
            for neighbor in self.neighborsOf(current):
                if neighbor not in visited:
                    frontier.add(neighbor)
    
    return HexRegion(region, self)


In [ ]:
region = grid.flood_fill(128)
print(f"Found {len(region)} hexes with style '{grid.hexes[128].style.name}'")
print(f"found {region.perimeterGraph()} hexes on the perimeter")
print(region.trace_boundary_v2(debug=True))

Found 144 hexes with style 'HexStyle'
found {} hexes on the perimeter
trace_boundary_v2
[]


In [ ]:
region.outside(), region.dangerous_outside()

(HexRegion(hexes=set(), hex_grid=<HexMagic.primitives.HexGrid object>),
 HexRegion(hexes=set(), hex_grid=<HexMagic.primitives.HexGrid object>))

In [ ]:
??HexRegion.outside


```python
@patch
def outside(self:HexRegion,ring=1):
    m = set()
    grid = self.hex_grid
    total = grid.nCols * grid.nRows
    for index in self.hexes:
        ring_hexpositions = HexPosition(0, 0, 0).ring(ring)
        neighbor_indices = [grid.hexposition_to_index(hp, index) for hp in ring_hexpositions]
        for neighbor in neighbor_indices:
            if neighbor >= 0 and neighbor < total and neighbor not in self.hexes:
                m.add(neighbor)
    return HexRegion(m,grid) # Filter out-of-bounds
```

**File:** `~/HexMagic/HexMagic/primitives.py`

my perimeter hex is broken

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The issue is that `perimeterHexes` relies on `outside()`, which only finds hexes **inside the grid** that are not in the region. When your region is the entire grid, `outside()` returns empty because there's nothing outside.

For a region touching the grid boundary, you need to also detect hexes that have **off-grid neighbors** (neighbor_idx < 0).

```python
@patch
def perimeterHexes(self: HexRegion) -> HexRegion:
    """Find hexes on the perimeter (have at least one non-region or off-grid neighbor)."""
    grid = self.hex_grid
    perimeter = set()
    
    for idx in self.hexes:
        for hp in HexPosition(0, 0, 0).ring(1):
            neighbor = grid.hexposition_to_index(hp, idx)
            # Perimeter if neighbor is off-grid OR not in region
            if neighbor < 0 or neighbor not in self.hexes:
                perimeter.add(idx)
                break
    
    return HexRegion(perimeter, grid)
```

This should now correctly identify edge hexes even when the region fills the entire grid.

In [ ]:
@patch
def flood_fill(self: HexGrid, start_idx: int) -> HexRegion:
    """Flood fill from start_idx to all connected hexes with same style."""
    if start_idx < 0 or start_idx >= len(self.hexes):
        return HexRegion(set(), self)
    
    target_style = self.hexes[start_idx].style.name
    visited = set()
    region = set()
    frontier = {start_idx}
    
    while frontier:
        current = frontier.pop()
        if current in visited:
            continue
        visited.add(current)
        
        if self.hexes[current].style.name == target_style:
            region.add(current)
            # Add all neighbors to frontier
            for neighbor in self.neighborsOf(current):
                if neighbor not in visited:
                    frontier.add(neighbor)
    
    return HexRegion(region, self)

In [ ]:
# Find all connected "Lowland" hexes starting from hex 128
region = grid.flood_fill(128)
print(f"Found {len(region)} hexes with style '{grid.hexes[128].style.name}'")
print(region.hexes)

We just have some corner cases in the map. hex 131 for instance

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

I see - hex 131 is at the corner of the map where multiple edges face off-grid. The issue is that we're jumping when we shouldn't be. 

The problem: checking `neighbor in perm_hexes.hexes` isn't quite right. Two perimeter hexes can be adjacent, but their shared edge might be *internal* to the region (not a boundary edge). We should only advance along *boundary* edges.

Here's the fix - check if the edge is a boundary edge (neighbor NOT in region), not whether neighbor is a perimeter hex:

```python
@patch
def trace_boundary_v2(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary using (hex_idx, vertex_idx) state."""
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    perm_hexes = self.perimeterHexes()
    
    # Single hex - return all 6 vertices
    if len(perm_hexes.hexes) == 1:
        idx = next(iter(perm_hexes.hexes))
        return [(idx, v) for v in range(6)]
    
    # Find starting hex and vertex (first boundary edge)
    start_hex = None
    start_vertex = None
    
    for idx in perm_hexes.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        next_vertex = (current_vertex + 1) % 6
        
        # Find which direction this edge faces
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            # Key: check if edge is BOUNDARY (neighbor outside region)
            is_boundary_edge = (neighbor < 0) or (neighbor not in self.hexes)
            
            if is_boundary_edge:
                # Boundary edge - stay on current hex, advance vertex
                current_vertex = next_vertex
            else:
                # Interior edge - jump to neighbor hex
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
        else:
            current_vertex = next_vertex
        
        # Back to start?
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        if len(path) > len(self.hexes) * 6:
            print(f"⚠ Warning: path too long")
            break
    
    return path
```

The key change: `is_boundary_edge = (neighbor < 0) or (neighbor not in self.hexes)` - we're checking if the edge faces *outside the region*, not whether the neighbor is a perimeter hex.

In [ ]:
@patch
def trace_boundary_v2(self: HexRegion, debug=False) -> list[tuple[int, int]]:
    """Trace boundary using (hex_idx, vertex_idx) state."""
    if not self.hexes:
        return []
    
    grid = self.hex_grid
    perm_hexes = self.perimeterHexes()
    
    # Single hex - return all 6 vertices
    if len(perm_hexes.hexes) == 1:
        idx = next(iter(perm_hexes.hexes))
        return [(idx, v) for v in range(6)]
    
    # Find starting hex and vertex (first boundary edge)
    start_hex = None
    start_vertex = None
    
    for idx in perm_hexes.hexes:
        for dir_idx, hp in enumerate(HexPosition(0, 0, 0).ring(1)):
            neighbor = grid.hexposition_to_index(hp, idx)
            if neighbor < 0 or neighbor not in self.hexes:
                start_hex = idx
                start_vertex = Hex._direction_to_vertices[dir_idx][0]
                break
        if start_hex is not None:
            break
    
    if start_hex is None:
        return []
    
    if debug:
        print(f"Starting at hex {start_hex}, vertex {start_vertex}")
    
    path = []
    current_hex = start_hex
    current_vertex = start_vertex
    
    while True:
        path.append((current_hex, current_vertex))
        
        next_vertex = (current_vertex + 1) % 6
        
        # Find which direction this edge faces
        edge_dir = None
        for dir_idx, (v1, v2) in enumerate(Hex._direction_to_vertices):
            if v1 == current_vertex and v2 == next_vertex:
                edge_dir = dir_idx
                break
        
        if edge_dir is not None:
            hp = HexPosition.directions()[edge_dir]
            neighbor = grid.hexposition_to_index(hp, current_hex)
            
            # Key: check if edge is BOUNDARY (neighbor outside region)
            is_boundary_edge = (neighbor < 0) or (neighbor not in self.hexes)
            
            if is_boundary_edge:
                # Boundary edge - stay on current hex, advance vertex
                current_vertex = next_vertex
            else:
                # Interior edge - jump to neighbor hex
                current_hex = neighbor
                current_vertex = (next_vertex + 3) % 6
        else:
            current_vertex = next_vertex
        
        # Back to start?
        if current_hex == start_hex and current_vertex == start_vertex:
            break
        
        if len(path) > len(self.hexes) * 6:
            print(f"⚠ Warning: path too long")
            break
    
    return path


In [ ]:
??HexGrid.hexposition_to_index


```python
@patch
def hexposition_to_index(self: HexGrid, hexpos: HexPosition, origin_index:int = None) -> int:
    if origin_index is None:
        origin_index = int(len(self.hexes)/2)
    """Convert HexPosition (relative to origin) back to grid index.
    Returns -1 if out of bounds."""
    # Get origin's cube coordinates
    origin_row, origin_col = self.index_to_row_col(origin_index)
    origin_q = origin_col - (origin_row - (origin_row & 1)) // 2
    origin_r = origin_row
    origin_s = -origin_q - origin_r

    # Add relative position to origin
    abs_q = hexpos.q + origin_q
    abs_r = hexpos.r + origin_r
    abs_s = hexpos.s + origin_s

    # Convert cube back to odd-r offset
    row = abs_r
    col = abs_q + (abs_r - (abs_r & 1)) // 2

    # Convert to index and check bounds
    return self.row_col_to_index(row, col)
```

**File:** `~/HexMagic/HexMagic/primitives.py`

In [ ]:
??HexGrid.row_col_to_index


```python
@patch
def row_col_to_index(self: HexGrid, row: int, col: int) -> int:
    """Convert (row, col) to flat grid index. Returns -1 if out of bounds."""
    if row < 0 or row >= self.nRows or col < 0 or col >= self.nCols:
        return -1
    return row * self.nCols + col
```

**File:** `~/HexMagic/HexMagic/primitives.py`

In [ ]:
@patch
def dangerous_hpi(self: HexGrid, hexpos: HexPosition, origin_index:int = None) -> int:
    if origin_index is None:
        origin_index = int(len(self.hexes)/2)
    """Convert HexPosition (relative to origin) back to grid index.
    Returns -1 if out of bounds."""
    # Get origin's cube coordinates
    origin_row, origin_col = self.index_to_row_col(origin_index)
    origin_q = origin_col - (origin_row - (origin_row & 1)) // 2
    origin_r = origin_row
    origin_s = -origin_q - origin_r

    # Add relative position to origin
    abs_q = hexpos.q + origin_q
    abs_r = hexpos.r + origin_r
    abs_s = hexpos.s + origin_s

    # Convert cube back to odd-r offset
    row = abs_r
    col = abs_q + (abs_r - (abs_r & 1)) // 2

    # Convert to index and check bounds
    return row * self.nCols + col

In [ ]:
@patch
def dangerous_outside(self:HexRegion,ring=1):
    m = set()
    grid = self.hex_grid
    total = grid.nCols * grid.nRows
    for index in self.hexes:
        ring_hexpositions = HexPosition(0, 0, 0).ring(ring)
        neighbor_indices = [grid.dangerous_hpi(hp, index) for hp in ring_hexpositions]
        for neighbor in neighbor_indices:
            if neighbor >= 0 and neighbor < total and neighbor not in self.hexes:
                m.add(neighbor)
    return HexRegion(m,grid) 

In [ ]:
@patch
def perimeterHexes(self:HexRegion):
    return self & (self.dangerous_outside().dangerous_outside())